In [37]:
TAG: str = "baseline-kmeans"
DATASET: str = "ColorectalCarcinoma"
REPETITIONS: int = 5

In [38]:
import sys
sys.path.append("../")

In [39]:
import numpy as np
import tqdm
import torch
from torch.utils import data
from utils.load_dataset import load_dataset
from evaluation import evaluation
import os
from sklearn.cluster import KMeans

In [40]:
import pandas as pd

In [41]:
train_dataset, test_dataset = load_dataset(DATASET)

dataset = data.ConcatDataset([train_dataset, test_dataset])
class_num = len(train_dataset.tensors[1].unique())
X_shape = train_dataset.tensors[0].shape[1]

X = np.concatenate((dataset.datasets[0].tensors[0].numpy(),  dataset.datasets[1].tensors[0].numpy()), axis=0)
y = np.concatenate((dataset.datasets[0].tensors[1].numpy(),  dataset.datasets[1].tensors[1].numpy()), axis=0)


Found cached dataset colorectal-carcinoma-microbiome-fengq (/home/MCB/wwydmanski/.cache/huggingface/datasets/wwydmanski___colorectal-carcinoma-microbiome-fengq/presence-absence/0.0.0/d342abdda5d78952841e239b73a3d87b2d7d70e0093401b57862bacb5b8ff86c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
class_num, X.shape

(2, (107, 606))

## Initialize network

## Implement Barlow Twins loss

## Prepare clustering evaluation

In [43]:
def cluster(model, X, y):
    y_pred = model.predict(X)
    nmi, ari, f, acc = evaluation.evaluate(y, y_pred, class_num)
    return acc

## Train the model

In [44]:
final_accs = []
for rep in range(REPETITIONS):
    model = KMeans(n_clusters=class_num, random_state=rep).fit(X)
    acc = cluster(model, X, y)
    final_accs.append(acc)
    experiment.log_metric("acc", acc)

/home/MCB/wwydmanski/miniconda3/envs/img_processing/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/MCB/wwydmanski/miniconda3/envs/img_processing/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/MCB/wwydmanski/miniconda3/envs/img_processing/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/MCB/wwydmanski/miniconda3/envs/img_processing/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 

In [45]:
# Create `DATASET` folder if it doesn't exist
if not os.path.exists(f"results/{DATASET}"):
    os.makedirs(f"results/{DATASET}")

with open(f"results/{DATASET}/{TAG}.txt", "w") as f:
    f.write(str(round(np.mean(final_accs), 3)) + "~" + str(round(np.std(final_accs), 3)))